<a href="https://colab.research.google.com/github/Akshad135/MultiAgent/blob/main/MultiAgent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Figuring dependencies

## Installing Dependencies

In [1]:
!pip install \
  langchain \
  langchain-core \
  langchain-tavily \
  langchain-community \
  tavily-python \
  langgraph \
  --quiet

## Import Statements

In [2]:
import os
import json
import time
import requests
from typing import Any, Dict, List, Optional, Mapping
from langchain_core.tools import Tool
from langchain.llms.base import BaseLLM
from langchain.tools.tavily_search import TavilySearchResults
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnableSequence
from langchain_core.language_models import LLM
from langchain_core.callbacks import CallbackManagerForLLMRun
from langgraph.graph import StateGraph, END
from typing import TypedDict, Literal
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.callbacks.manager import CallbackManager
from google.colab import userdata

# Setting up agents

### SETUP

In [3]:
callbacks = [StreamingStdOutCallbackHandler()]
callback_manager = CallbackManager(callbacks)

### ENVs

In [4]:
MAX_RETRIES = 3

TAVILY_API_KEY = userdata.get('TAVILY_API_KEY')
os.environ["TAVILY_API_KEY"] = TAVILY_API_KEY

GROQ_API_KEY = userdata.get('GROQ_API_KEY')
GROQ_API_URL = "https://api.groq.com/openai/v1/chat/completions"

## Tavily setup

In [5]:
tavily_search = TavilySearchResults(k=5)
research_tool = Tool(
    name="tavily_search",
    description="Use for factual web search via Tavily",
    func=tavily_search.run,
)

## Groq Setup

In [6]:
class GroqLLM(LLM):
    model_name: str = "llama-3.1-8b-instant"
    temperature: float = 0.7
    max_tokens: int = 1024

    @property
    def _llm_type(self) -> str:
        return "groq"

    def _call(
        self,
        prompt: str,
        stop: Optional[List[str]] = None,
        run_manager: Optional[CallbackManagerForLLMRun] = None,
        **kwargs: Any,
    ) -> str:
        headers = {
            "Authorization": f"Bearer {GROQ_API_KEY}",
            "Content-Type": "application/json"
        }

        data = {
            "model": self.model_name,
            "messages": [
                {"role": "user", "content": prompt}
            ],
            "temperature": kwargs.get("temperature", self.temperature),
            "max_tokens": kwargs.get("max_tokens", self.max_tokens)
        }

        if stop:
            data["stop"] = stop

        try:
            response = requests.post(GROQ_API_URL, headers=headers, json=data)
            response.raise_for_status()
            result = response.json()
            return result["choices"][0]["message"]["content"]
        except Exception as e:
            return f"Error: {str(e)}"

    def _identifying_params(self) -> Mapping[str, Any]:
        return {
            "model_name": self.model_name,
            "temperature": self.temperature,
            "max_tokens": self.max_tokens
        }

### Creating instances

In [7]:
research_llm = GroqLLM(
    model_name="llama-3.1-8b-instant",
    temperature=0.5,
    max_tokens=512
)

answer_drafting_llm = GroqLLM(
    model_name="llama-3.1-8b-instant",
    temperature=0.7,
    max_tokens=1024
)

## State Definitions

In [8]:
class MyGraphState(TypedDict):
    user_query: str
    snippets: list[dict]
    verdict: str

class DraftState(MyGraphState, total=False):
    retry_count: int
    answer: str

### Model Prompts

In [20]:
fetch_prompt = """
You are a Research Assistant.
Call the function 'tavily_search' with this question: {query}
"""

evaluate_prompt = """
You are a Quality Evaluator.
Given these snippets for the question: {query}

Snippets:
{snippets}

Your task is to determine if ANY of these snippets contain relevant information that could help answer the question.
Respond with ONLY 'yes' if ANY snippet contains relevant information.
Respond with ONLY 'no' if NONE of the snippets contain relevant information.
"""

draft_prompt = """
You are an Answer Drafting Agent.

USER QUERY: {query}

INFORMATION SNIPPETS:
{snippets}

INSTRUCTIONS:
1. Use ONLY the information in the snippets above to answer the query
2. Write a comprehensive answer in paragraph form and not in point wise form.
3. Include the most important facts and recent breakthroughs mentioned in the snippets
4. Keep your answer factual and concise
5. Do NOT include any JSON formatting in your answer
"""

## Prompt Templates

In [21]:
fetch_prompt_template = PromptTemplate.from_template(fetch_prompt)
evaluate_prompt_template = PromptTemplate.from_template(evaluate_prompt)
draft_prompt_template = PromptTemplate.from_template(draft_prompt)

### Prompt Chains

In [22]:
fetch_chain = fetch_prompt_template | research_llm
evaluate_chain = evaluate_prompt_template | research_llm
draft_chain = draft_prompt_template | answer_drafting_llm

# Main Architecture

## Crawl Web using Tavily

In [23]:
def fetch_snippets(state: DraftState) -> DraftState:
    rc = state.get("retry_count", 0)
    print(f"[FETCH] Searching for: '{state['user_query']}' (attempt {rc+1}/{MAX_RETRIES+1})")

    try:
        raw = research_tool.run(state["user_query"])
        snippets = []

        if isinstance(raw, str):
            try:
                data = json.loads(raw)
            except json.JSONDecodeError:
                data = [{"text": raw}]
        else:
            data = raw

        if isinstance(data, dict):
            snippet_list = [data]
        elif isinstance(data, list):
            snippet_list = data
        else:
            snippet_list = [{"text": str(data)}]

        for idx, item in enumerate(snippet_list):
            if isinstance(item, dict):
                text = None
                for key in ["content", "text", "snippet"]:
                    if key in item and item[key]:
                        text = item[key]
                        break

                if not text:
                    text = str(item)

                if isinstance(text, str) and text.startswith("{") and "content" in text:
                    try:
                        parsed = json.loads(text.replace("'", "\""))
                        if "content" in parsed:
                            text = parsed["content"]
                    except:
                        pass

                snippets.append({"id": idx, "text": text})
            else:
                snippets.append({"id": idx, "text": str(item)})

        print(f"[FETCH] Found {len(snippets)} snippets")
        if snippets:
            preview = snippets[0]["text"][:150] + "..." if len(snippets[0]["text"]) > 150 else snippets[0]["text"]
            print(f"[FETCH] Sample: \"{preview}\"")

        verdict = "yes" if snippets else "no"

    except Exception as e:
        snippets = []
        verdict = "no"
        print(f"[FETCH] Error occurred during search")

    return {
        "user_query": state["user_query"],
        "snippets": snippets,
        "verdict": verdict,
        "retry_count": rc
    }

## Evaluate the fetch results

In [24]:
def evaluate_snippets(state: DraftState) -> DraftState:
    print(f"[EVALUATE] Analyzing {len(state['snippets'])} snippets for relevance")

    if not state["snippets"] or len(state["snippets"]) == 0:
        return {**state, "verdict": "no"}

    formatted_snippets = []
    for snippet in state["snippets"]:
        snippet_text = snippet["text"][:500] + "..." if len(snippet["text"]) > 500 else snippet["text"]
        formatted_snippets.append(f"Snippet {snippet['id']+1}: {snippet_text}")

    snippet_input = "\n\n".join(formatted_snippets)

    try:
        formatted_prompt = evaluate_prompt_template.format(
            query=state["user_query"],
            snippets=snippet_input
        )

        verdict = research_llm._call(formatted_prompt).strip().lower()

        if "yes" in verdict:
            verdict = "yes"
        else:
            verdict = "no"

    except Exception as e:
        verdict = state["verdict"]

    print(f"[EVALUATE] Verdict: {verdict.upper()}")
    return {**state, "verdict": verdict}

## Decide the next steps

In [25]:
def decide_next_step(state: DraftState) -> Literal["fetch_snippets", "ingest_snippets"]:
    rc = state.get("retry_count", 0)

    if state["verdict"].lower().startswith("no") and rc < MAX_RETRIES:
        print(f"[DECIDE] Insufficient information, retrying search")
        return "fetch_snippets"

    if state["verdict"].lower().startswith("no"):
        print(f"[DECIDE] Max retries reached, proceeding with available data")

    print(f"[DECIDE] Sufficient information found, proceeding to draft")
    return "ingest_snippets"

## Ingest the snippets

In [26]:
def ingest_snippets(state: DraftState) -> DraftState:
    print(f"[INGEST] Processing {len(state['snippets'])} snippets for drafting")
    return {**state, "retry_count": state.get("retry_count", 0) + 1}

## Draft final answer

In [27]:
def draft_answer(state: DraftState) -> DraftState:
    print(f"[DRAFT] Generating comprehensive response")

    formatted_snippets = []
    for snippet in state["snippets"]:
        snippet_text = snippet["text"][:500] + "..." if len(snippet["text"]) > 500 else snippet["text"]
        formatted_snippets.append(f"Snippet {snippet['id']+1}: {snippet_text}")

    snippet_input = "\n\n".join(formatted_snippets)

    try:
        formatted_prompt = draft_prompt_template.format(
            query=state["user_query"],
            snippets=snippet_input
        )

        answer = answer_drafting_llm._call(formatted_prompt)

    except Exception as e:
        answer = f"Unable to generate answer due to technical difficulties."

    print(f"[DRAFT] Response generated ({len(answer)} chars)")
    return {**state, "answer": answer}

## Creating the graph

In [28]:
graph = StateGraph(DraftState)
graph.add_node("fetch_snippets", fetch_snippets)
graph.add_node("evaluate_snippets", evaluate_snippets)
graph.add_node("decide_next_step", decide_next_step)
graph.add_node("ingest_snippets", ingest_snippets)
graph.add_node("draft_answer", draft_answer)

graph.set_entry_point("fetch_snippets")
graph.add_edge("fetch_snippets", "evaluate_snippets")
graph.add_conditional_edges(
    "evaluate_snippets",
    decide_next_step,
    {
        "fetch_snippets": "fetch_snippets",
        "ingest_snippets": "ingest_snippets"
    }
)
graph.add_edge("ingest_snippets", "draft_answer")
graph.add_edge("draft_answer", END)

app = graph.compile()

## Final answer

In [29]:
def run_research(query):
    initial_state = {"user_query": query, "retry_count": 0}
    print(f"\nRESEARCH QUERY: {query}")
    print("=" * 50)

    final_state = app.invoke(initial_state)

    print("=" * 50)
    print("\nFINAL ANSWER:")
    print(final_state.get("answer", "No answer was generated."))
    print("=" * 50)

    return final_state

In [33]:
if __name__ == "__main__":
    result = run_research("Tell me about the book 'Tuesdays with Morrie'?")


RESEARCH QUERY: Tell me about the book 'Tuesdays with Morrie'?
[FETCH] Searching for: 'Tell me about the book 'Tuesdays with Morrie'?' (attempt 1/4)
[FETCH] Found 5 snippets
[FETCH] Sample: "Tuesdays with Morrie: An Old Man, A Young Man and Life's Greatest Lesson is a 1997 memoir by American author Mitch Albom. The book is about a series o..."
[EVALUATE] Analyzing 5 snippets for relevance
[EVALUATE] Verdict: YES
[DECIDE] Sufficient information found, proceeding to draft
[INGEST] Processing 5 snippets for drafting
[DRAFT] Generating comprehensive response
[DRAFT] Response generated (1659 chars)

FINAL ANSWER:
'Tuesdays with Morrie: An Old Man, A Young Man and Life's Greatest Lesson' is a 1997 memoir by American author Mitch Albom. The book revolves around a series of visits Albom made to his former Brandeis University sociology professor, Morrie Schwartz, who was diagnosed with amyotrophic lateral sclerosis (ALS). As Morrie's condition progresses, he shares his wisdom and life lessons 